# This is to define model options for GEM-2

In [2]:
# magic lines that avoid re-start 
%load_ext autoreload
%autoreload 2
import csv, random, os
import pybamm as pb;import pandas as pd;import numpy as np;
import os, json,openpyxl,traceback,multiprocessing,scipy.optimize
import matplotlib.pyplot as plt;
import imageio,timeit,random,time, signal
from scipy.io import savemat,loadmat;
from pybamm import constants,exp;import matplotlib as mpl; 
fs=17; 
font = {'family' : 'DejaVu Sans','size'   : fs}
mpl.rc('font', **font)

In [3]:
parameter_names = [
    "Scan No",
    "Exp No.", # 2,3,5
    "Ageing temperature", # 10,25,40
    'Inner SEI lithium interstitial diffusivity [m2.s-1]',#1e-19~5e-18
    'Dead lithium decay constant [s-1]',
    'Lithium plating kinetic rate constant [m.s-1]',
    'Negative electrode LAM constant proportional term [s-1]',
    'Positive electrode LAM constant proportional term [s-1]',
    'Negative electrode cracking rate',
    'Outer SEI partial molar volume [m3.mol-1]',
    "SEI growth activation energy [J.mol-1]", # 1e4~3.8e4
    "Negative cracking growth activation energy [J.mol-1]",#0
    "Negative electrode diffusivity activation energy [J.mol-1]",#1.7e4
    "Positive electrode diffusivity activation energy [J.mol-1]",#1.2e4
    "Contact resistance [Ohm]",#0.010
    'Total heat transfer coefficient [W.m-2.K-1]',#20
    'Initial electrolyte excessive amount ratio', # 1.0 or 0.99
    "Ratio of lithium moles to SEI moles", # 2.0
]
para_short_name = [
    "Exp", # 2,3,5
    "T", # 10,25,40
    'Dint',#1e-19~5e-18
    'Decay',
    'k_LiP',
    'lam_ne',
    'lam_pe',
    'kcr_ne',
    'Vsei',
    "Eact_SEI", # 1e4~3.8e4
    "Eact_kcr_ne",#0
    "Eact_Ds_ne",#1.7e4
    "Eact_Ds_pe",#1.2e4
    "Rct",#0.010
    'h',#20
    'Elely res', # 1.0 or 0.99
    "z_sei", # 2.0
]
Dint_default = [5e-18,];    Decay_default = [1e-6,]
k_LiP_default = [1e-9,];    LAM_Ne_default = [3e-7,]
LAM_Pe_default = [3e-7,];   k_cr_Ne_default=[1e-20,]
V_SEI_default = [6.67e-5];  Eact_SEI_default=[1e4,]
Eact_cr_Ne_default=[0.0,];  Eact_Ds_Ne_default=[1.7e4,]
Eact_Ds_Pe_default=[1.2e4,];Rct_default=[0.010,]
Elely_res_default = [1.0,]; h_default = [20,]
Ratio_Li_SEI_default=[2.0,]

In [4]:
# old functions
def save_rows_to_csv(file_path, rows, header):
    with open(file_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(header)  # Write parameter names as the header row
        writer.writerows(rows)
def generate_combinations(Bounds, Num_tot):
    lower_bounds = []
    upper_bounds = []
    for bound in Bounds:
        lower_bounds.append(bound[0])
        upper_bounds.append(bound[1])
    combinations = []
    for _ in range(Num_tot):
        combination = []
        for lower, upper in zip(lower_bounds, upper_bounds):
            value = random.uniform(lower, upper)
            combination.append(value)
        combinations.append(combination)
    return combinations

def save_combinations_to_csv(combinations, parameter_names, filename):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(parameter_names)  # Write parameter names as the first row
        for combination in combinations:
            writer.writerow(combination)

In [5]:
BasicPath =  os.path.expanduser("~/EnvPBGEM_Linux/SimSave/P2_R9_Dim")
Target  = f'/Get_Random_sets/'
if not os.path.exists(BasicPath + Target):
   os.mkdir(BasicPath + Target)

In [14]:
def Get_Scan_files(
        BasicPath,Target_name,model_options,
        parameter_names,para_short_name,
        Pack,
        rows_per_file,Bundle):
    
    import itertools
    para_dict_Same = {
        "Cycles within RPT":1,
        "RPT temperature":25,
        "Mesh list":[15,15,15,80,50],  
        "Para_Set": "OKane2023",
        "Model option":model_options,
        "Current solvent concentration in the reservoir [mol.m-3]":4541.0,
        "Current electrolyte concentration in the reservoir [mol.m-3]":1000,
        "Ratio of Li-ion concentration change in " 
        "electrolyte consider solvent consumption":1.0,
        'EC initial concentration in electrolyte [mol.m-3]':4541.0,
        'Typical EC concentration in electrolyte [mol.m-3]':4541.0, 
        "Negative electrode number of cracks per unit area [m-2]": 3.18e15,
        "Initial inner SEI thickness [m]": 1.23625e-08,
        "Initial outer SEI thickness [m]": 1.23625e-08,
        "Negative electrode porosity": 0.222393,
        }
    unchange_key2 = list(para_dict_Same.keys())
    unchange_val2 = list(para_dict_Same.values())
    short_pack = [lst for lst in Pack if len(lst) > 1]
    selected_indices = [i for i, lst in enumerate(Pack) if len(lst) > 1]
    shortList_para_short_name = [para_short_name[i] for i in selected_indices]
    shortList_para_short_name.insert(0,"No")
    really_change_val =  [
        list(comb) for comb in itertools.product(*short_pack)]

    change_val =  [
        list(comb) for comb in itertools.product(*Pack)]
    combinations = [[i+1,*elem, *unchange_val2] for i,elem in enumerate(change_val)]
    comb_short   = [[i+1,*elem] for i,elem in enumerate(really_change_val)]
    parameter_names = [*parameter_names,*unchange_key2]
    print("Total cases number is",len(combinations))
    if Bundle:
        # Specify the total number of cases
        total_cases = len(combinations)
        # Specify the number of rows per CSV file, rows_per_file
        # Calculate the number of files needed
        num_files = (total_cases - 1) // rows_per_file + 1
        # Create the target folder
        folder_path = os.path.join(BasicPath, "Get_Random_sets", Target_name)
        os.makedirs(folder_path, exist_ok=True)
        # Write data to each CSV file
        for i in range(num_files):
            file_name = f"Bundle_{i+1}.csv"
            file_path = os.path.join(folder_path, file_name)
            start_row = i * rows_per_file
            end_row = min(start_row + rows_per_file, total_cases)
            rows = combinations[start_row:end_row]
            save_rows_to_csv(file_path, rows, parameter_names)
        filename = BasicPath+f"/Get_Random_sets/{Target_name}/"+f'{Target_name}.csv'
        filename_short = BasicPath+f"/Get_Random_sets/{Target_name}/"+f'{Target_name}_s.csv'
        save_combinations_to_csv(combinations, parameter_names, filename)
        save_combinations_to_csv(comb_short, shortList_para_short_name, filename_short)
        print(f"Combinations saved to '{Target_name}.csv' file.") 
        print(f"CSV files created in folder '{Target_name}'.")
    else:
        filename = BasicPath+"/Get_Random_sets/"+f'{Target_name}.csv'
        save_combinations_to_csv(combinations, parameter_names, filename)
        print(f"Combinations saved to '{Target_name}.csv' file.") 
    return len(combinations)


In [7]:
def get_list_from_tuple(d, num):
    if d[1] > d[0]:
        if d[1] > 100 * d[0]:
            result_list = (np.exp(np.linspace(np.log(d[0]), np.log(d[1]), num=num))).tolist()
        else:
            result_list = (np.linspace(d[0], d[1], num=num)).tolist()
    else:
        result_list = []
    return result_list
def Get_Scan_Orth_Latin(
        BasicPath,Target_name,model_options,
        parameter_names,para_short_name,
        Pack, num,
        rows_per_file,Bundle):
    
    import itertools; from pyDOE import lhs
    para_dict_Same = {
        "Cycles within RPT":1,
        "RPT temperature":25,
        "Mesh list":[5,5,5,60,20],  
        "Para_Set": "OKane2023",
        "Model option":model_options,
        "Current solvent concentration in the reservoir [mol.m-3]":4541.0,
        "Current electrolyte concentration in the reservoir [mol.m-3]":1000,
        "Ratio of Li-ion concentration change in " 
        "electrolyte consider solvent consumption":1.0,
        'EC initial concentration in electrolyte [mol.m-3]':4541.0,
        'Typical EC concentration in electrolyte [mol.m-3]':4541.0, 
        "Negative electrode number of cracks per unit area [m-2]": 3.18e15,
        "Initial inner SEI thickness [m]": 1.23625e-08,
        "Initial outer SEI thickness [m]": 1.23625e-08,
        "Negative electrode porosity": 0.222393,
        }
    unchange_key2 = list(para_dict_Same.keys())
    unchange_val2 = list(para_dict_Same.values())
    
    Pack_tuple = []; Pack_tuple_index = []
    Pack_list = [];  Pack_list_index  = []
    for i,item in enumerate(Pack):
        if isinstance(item, tuple):
            Pack_tuple.append(item)
            Pack_tuple_index.append(i)
        elif isinstance(item, list):
            Pack_list.append(item)
            Pack_list_index.append(i)
    com_tuple = []; comb_tu_list =[]
    if len(Pack_tuple) > 1:
        for tuple_i in Pack_tuple:
            com_tuple.append( get_list_from_tuple(tuple_i, num) )
        # apply Latin Hypercube:
        #print(com_tuple)
        samples = lhs(len(com_tuple), samples=num)
        for sample in samples:
            combination = []
            for i, candidate_list in enumerate(com_tuple):
                index = int(sample[i] * num)
                combination.append(candidate_list[index])
            comb_tu_list.append(combination)
    else:
        print("error! Pack_tuple must has 2 elements")
    # apply product sampling:
    comb_li_list = [list(comb) for comb in itertools.product(*Pack_list)]
    #print(comb_tu_list)
    #print(comb_li_list)
    Big_Comb = []
    for comb_tu in comb_tu_list:
        for comb_li in comb_li_list:
            big_comb = [0] * (len(comb_tu)+len(comb_li))
            for comb_tu_i,index in zip(comb_tu,Pack_tuple_index):
                big_comb[index] = comb_tu_i
            for comb_li_i,index in zip(comb_li,Pack_list_index):
                big_comb[index] = comb_li_i
            Big_Comb.append(big_comb)
    #print(Big_Comb)
    Big_Comb
    combinations = [[i+1,*elem, *unchange_val2] for i,elem in enumerate(Big_Comb)]

    parameter_names = [*parameter_names,*unchange_key2]
    print("Total cases number is",len(combinations))
    if Bundle:
        # Specify the total number of cases
        total_cases = len(combinations)
        # Specify the number of rows per CSV file, rows_per_file
        # Calculate the number of files needed
        num_files = (total_cases - 1) // rows_per_file + 1
        # Create the target folder
        folder_path = os.path.join(BasicPath, "Get_Random_sets", Target_name)
        os.makedirs(folder_path, exist_ok=True)
        # Write data to each CSV file
        for i in range(num_files):
            file_name = f"Bundle_{i+1}.csv"
            file_path = os.path.join(folder_path, file_name)
            start_row = i * rows_per_file
            end_row = min(start_row + rows_per_file, total_cases)
            rows = combinations[start_row:end_row]
            save_rows_to_csv(file_path, rows, parameter_names)
        filename = BasicPath+f"/Get_Random_sets/{Target_name}/"+f'{Target_name}.csv'
        filename_short = BasicPath+f"/Get_Random_sets/{Target_name}/"+f'{Target_name}_s.csv'
        save_combinations_to_csv(combinations, parameter_names, filename)
        # save_combinations_to_csv(comb_short, shortList_para_short_name, filename_short)
        print(f"Combinations saved to '{Target_name}.csv' file.") 
        print(f"CSV files created in folder '{Target_name}'.")
    else:
        filename = BasicPath+"/Get_Random_sets/"+f'{Target_name}.csv'
        save_combinations_to_csv(combinations, parameter_names, filename)
        print(f"Combinations saved to '{Target_name}.csv' file.") 
    return len(combinations)


# SEI only

In [17]:
options_SEI = {
    "SEI": "interstitial-diffusion limited",
    "SEI on cracks": "true",
    
    "lithium plating": "none",
    "lithium plating porosity change":"false",
    "particle mechanics": "constant cracks",
    "loss of active material": "none",

    "contact resistance": "true",
    "open-circuit potential": "current sigmoid",
    "SEI film resistance": "distributed", 
    "SEI porosity change": "true",
    "thermal": "lumped",
}
Exps  = [2,3]
Temps = [10.0,25.0,40.0,]
cands_Dint  = [2.68e-18]
cands_V_SEI = [4e-5] # previous good: 5e-18 and 4e-5 or 8e-18 and 7e-5
cands_Elely_res = [0.99,]
cands_Eact_SEI= [1.66e4]
Eact_cr_Ne_default=[0.0,];  cands_Eact_Ds_Ne = [1.56e4]
cands_Eact_Ds_Pe=[1.18e4]; cands_h = [17,]
Pack = [
    Exps,Temps,
    cands_Dint,    
    Decay_default, k_LiP_default,LAM_Ne_default,
    LAM_Pe_default,k_cr_Ne_default,
    cands_V_SEI,
    cands_Eact_SEI,Eact_cr_Ne_default,
    cands_Eact_Ds_Ne,cands_Eact_Ds_Pe,
    Rct_default,cands_h,
    cands_Elely_res,
    Ratio_Li_SEI_default
]
Target_name = "SEI_Exp23_Paper"
rows_per_file = 3; 
num = 50
num_scans = Get_Scan_files(
        BasicPath,Target_name,options_SEI,
        parameter_names,para_short_name,
        Pack, 
        rows_per_file,Bundle=True)

Total cases number is 6
Combinations saved to 'SEI_Exp23_Paper.csv' file.
CSV files created in folder 'SEI_Exp23_Paper'.


# SEI_Dry

In [16]:
"""  previous run: SEI_Dryout_230704_2
P2_R9_Dim\SEI_Dryout_230704
for the first round, SEI_Dryout_230704, SEI-on-crack is false, then
Dint=5e-18 fit pretty well. but after it becomes true in 
SEI_Dryout_230704_2, the capacity loss is too quick!"""
options_SEI_Dry = {
    "SEI": "interstitial-diffusion limited",
    "SEI on cracks": "true",
    
    "lithium plating": "none",
    "lithium plating porosity change":"false",
    "particle mechanics": "constant cracks",
    "loss of active material": "none",

    "contact resistance": "true",
    "open-circuit potential": "current sigmoid",
    "SEI film resistance": "distributed", 
    "SEI porosity change": "true",
    "thermal": "lumped",
}
Exps  = [2,3,]
Temps = [10.0,25.0,40.0,]
cands_Dint  = [4.5e-19]
cands_V_SEI = [4e-5] # previous good: 5e-18 and 4e-5 or 8e-18 and 7e-5
cands_Elely_res = [1.0,]
Eact_SEI_default=[1e4,]
Eact_cr_Ne_default=[0.0,];  Eact_Ds_Ne_default = [6e4,]
cands_Eact_Ds_Pe=[1.2e4,]
Pack = [
    Exps,Temps,
    cands_Dint,    
    Decay_default, k_LiP_default,LAM_Ne_default,
    LAM_Pe_default,k_cr_Ne_default,
    cands_V_SEI,
    Eact_SEI_default,Eact_cr_Ne_default,
    Eact_Ds_Ne_default,cands_Eact_Ds_Pe,
    Rct_default,h_default,
    cands_Elely_res,
    Ratio_Li_SEI_default
]
Target_name = "SEI_Dry_Exp23_Paper"
rows_per_file = 3; 
num_scans = Get_Scan_files(
    BasicPath,Target_name,options_SEI_Dry,
    parameter_names,para_short_name,Pack,
    rows_per_file,Bundle=True)


Total cases number is 6
Combinations saved to 'SEI_Dry_Exp23_Paper.csv' file.
CSV files created in folder 'SEI_Dry_Exp23_Paper'.


In [20]:
# new: use Get_Scan_Orth_Latin to combine Latin with orthogonal 
# Note: for Latin, must has two tuples at least!
options_SEI_Dry = {
    "SEI": "interstitial-diffusion limited",
    "SEI on cracks": "true",
    
    "lithium plating": "none",
    "lithium plating porosity change":"false",
    "particle mechanics": "constant cracks",
    "loss of active material": "none",

    "contact resistance": "true",
    "open-circuit potential": "current sigmoid",
    "SEI film resistance": "distributed", 
    "SEI porosity change": "true",
    "thermal": "lumped",
}
Exps  = [2,]
Temps = [10.0,25.0,40.0,]
cands_Dint  = (1e-19, 5e-18)
cands_V_SEI =  (3e-5,10e-5)
cands_Elely_res = [1.0,]
Eact_SEI_default=[1e4,]
Eact_cr_Ne_default=[0.0,];  Eact_Ds_Ne_default = [6e4,]
cands_Eact_Ds_Pe=[1.2e4,]
Pack = [
    Exps,Temps,
    cands_Dint,    
    Decay_default, k_LiP_default,LAM_Ne_default,
    LAM_Pe_default,k_cr_Ne_default,
    cands_V_SEI,
    Eact_SEI_default,Eact_cr_Ne_default,
    Eact_Ds_Ne_default,cands_Eact_Ds_Pe,
    Rct_default,h_default,
    cands_Elely_res,
    Ratio_Li_SEI_default
]
Target_name = "SEI_Dry_6"
rows_per_file = 4; 
num = 20
num_scans = Get_Scan_Orth_Latin(
        BasicPath,Target_name,options_SEI_Dry,
        parameter_names,para_short_name,
        Pack, num,
        rows_per_file,Bundle=True)

[[1e-19, 3.5789473684210526e-19, 6.157894736842105e-19, 8.736842105263159e-19, 1.1315789473684211e-18, 1.3894736842105263e-18, 1.6473684210526317e-18, 1.9052631578947366e-18, 2.163157894736842e-18, 2.4210526315789474e-18, 2.6789473684210524e-18, 2.936842105263158e-18, 3.1947368421052632e-18, 3.4526315789473683e-18, 3.710526315789474e-18, 3.9684210526315795e-18, 4.2263157894736845e-18, 4.4842105263157895e-18, 4.742105263157895e-18, 5e-18], [3e-05, 3.368421052631579e-05, 3.736842105263158e-05, 4.105263157894737e-05, 4.473684210526316e-05, 4.842105263157895e-05, 5.2105263157894744e-05, 5.578947368421053e-05, 5.947368421052632e-05, 6.315789473684212e-05, 6.68421052631579e-05, 7.05263157894737e-05, 7.421052631578948e-05, 7.789473684210527e-05, 8.157894736842107e-05, 8.526315789473685e-05, 8.894736842105265e-05, 9.263157894736843e-05, 9.631578947368422e-05, 0.0001]]
[[2.936842105263158e-18, 3.368421052631579e-05], [3.4526315789473683e-18, 4.473684210526316e-05], [3.1947368421052632e-18, 4.10

# SEIcr_Dry

In [ ]:
# SEI, cracking, dry-out only, previous run: no
options_SEIcr_Dry = {
    "SEI": "interstitial-diffusion limited",
    "SEI on cracks": "true",
    "lithium plating": "none",
    "lithium plating porosity change":"false",
    "particle mechanics": ("swelling and cracking", "swelling only"),
    "loss of active material": "none",
    "contact resistance": "true",
    "open-circuit potential": "current sigmoid",
    "SEI film resistance": "distributed", 
    "SEI porosity change": "true",
    "thermal": "lumped",
}
Elely_res = 1.0

# SEI_LAM

In [ ]:
# SEI+ stress-driven LAM only, previous run: no, 
options_SEI_LAM = {
    "SEI": "interstitial-diffusion limited",
    "SEI on cracks": "true",

    "lithium plating": "none",
    "lithium plating porosity change":"false",
    "particle mechanics": "constant cracks",
    "loss of active material": "stress-driven",

    "contact resistance": "true",
    "open-circuit potential": "current sigmoid",
    "SEI film resistance": "distributed", 
    "SEI porosity change": "true",
    "thermal": "lumped",
}
Elely_res = 0.99

# SEIcr_LAM

In [ ]:
# SEI+ stress-driven LAM, cracking, previous run: no, 
options_SEIcr_LAM = {
    "SEI": "interstitial-diffusion limited",
    "SEI on cracks": "true",

    "lithium plating": "none",
    "lithium plating porosity change":"false",
    "particle mechanics": ("swelling and cracking", "swelling only"),
    "loss of active material": "stress-driven",

    "contact resistance": "true",
    "open-circuit potential": "current sigmoid",
    "SEI film resistance": "distributed", 
    "SEI porosity change": "true",
    "thermal": "lumped",
}
Elely_res = 0.99

# Almost all (old)

In [21]:
options_Almost_all = {
    "SEI": "interstitial-diffusion limited",
    "SEI on cracks": "true",

    "lithium plating": "partially reversible",
    "lithium plating porosity change":"true", # use to be false in old
    "particle mechanics": ("swelling and cracking", "swelling only"),
    "loss of active material": ("stress-driven","none"),
    
    "contact resistance": "true",
    "open-circuit potential": "current sigmoid",
    "SEI film resistance": "distributed", 
    "SEI porosity change": "true",
    "thermal": "lumped",
}
Exps  = [2,]
Temps = [10.0,25.0,40.0,]
cands_Dint  = [5e-18,8e-18]
cands_V_SEI = [4e-5,7e-5] 
cands_Elely_res = [1.0,]
Decay_default=[1.19E-13]
k_LiP_default=[4.5e-15]
LAM_Ne_default=[2.77e-16]
LAM_Pe_default = [1e-16]
k_cr_Ne_default=[5.29e-25]
Eact_SEI_default=[1e4]
Eact_cr_Ne_default=[0.0,]
Eact_Ds_Ne_default=[1.7e4,]
Eact_Ds_Pe_default=[1.2e4,]
Rct_default=[0.010,]
h_default= [20,]
Ratio_Li_SEI_default=[2.0,]
Pack = [
    Exps,Temps,
    cands_Dint,    
    Decay_default, k_LiP_default,LAM_Ne_default,
    LAM_Pe_default,k_cr_Ne_default,
    cands_V_SEI,
    Eact_SEI_default,Eact_cr_Ne_default,
    Eact_Ds_Ne_default,Eact_Ds_Pe_default,
    Rct_default,h_default,
    cands_Elely_res,
    Ratio_Li_SEI_default
] 
Target_name = "Almost_all_refine"
rows_per_file = 6; 
num_scans = Get_Scan_files(
    BasicPath,Target_name,options_Almost_all,
    parameter_names,Pack,
    rows_per_file,Bundle=True)


TypeError: Get_Scan_files() missing 1 required positional argument: 'rows_per_file'

# SEIcr_LAM_Dry

In [ ]:
# SEI+ stress-driven LAM, cracking, dry-out
options_SEIcr_LAM_Dry = {
    "SEI": "interstitial-diffusion limited",
    "SEI on cracks": "true",

    "lithium plating": "none",
    "lithium plating porosity change":"false",
    "particle mechanics": ("swelling and cracking", "swelling only"),
    "loss of active material": "stress-driven",

    "contact resistance": "true",
    "open-circuit potential": "current sigmoid",
    "SEI film resistance": "distributed", 
    "SEI porosity change": "true",
    "thermal": "lumped",
}
Elely_res = 1.0

# SEI_Dry_LiP (Dryout_noLAM)

In [ ]:
""" # Previous run: Dryout_noLAM 
# good fit:
\\wsl.localhost\Ubuntu-22.04\home\ruihe_li\EnvPBGEM_Linux\SimSave\P2_R9_Dim\Dryout_noLAM
case 6,12,18
"""
options_SEI_Dry_LiP = {
    "SEI": "interstitial-diffusion limited",
    "SEI on cracks": "true",

    "lithium plating": "partially reversible",
    "lithium plating porosity change":"true",
    "particle mechanics": "constant cracks", 
    "loss of active material": "none",

    "contact resistance": "true",
    "open-circuit potential": "current sigmoid",
    "SEI film resistance": "distributed", 
    "SEI porosity change": "true",
    "thermal": "lumped",
}
Elely_res = 1.0

# SEI_LAM_LiP (noDryout_LAM)

In [30]:
# Previous run: noDryout_LAM 
options_SEI_LAM_LiP = {
    "SEI": "interstitial-diffusion limited",
    "SEI on cracks": "true",

    "lithium plating": "partially reversible",
    "lithium plating porosity change":"true",
    "particle mechanics": "constant cracks", 
    "loss of active material": "stress-driven",

    "contact resistance": "true",
    "open-circuit potential": "current sigmoid",
    "SEI film resistance": "distributed", 
    "SEI porosity change": "true",
    "thermal": "lumped",
}
Exps  = [2,]
Temps = [25.0,]
cands_Dint  = [3e-19,]
cands_Decay =  [1E-6,]
cands_LiP =  [1e-10]
num = 9;
cands_LAM_Ne = [2e-8]
cands_LAM_Pe = np.linspace(4e-6,5e-4, num=num)
cands_k_cr_Ne=[1e-25]
cands_V_SEI = [10e-5]
cands_Eact_Ds_Ne = [1.7e4]
cands_Elely_res = [0.99,]
Pack = [
    Exps,Temps,
    cands_Dint,    
    cands_Decay, cands_LiP,cands_LAM_Ne,
    cands_LAM_Pe,cands_k_cr_Ne,
    cands_V_SEI,
    Eact_SEI_default,Eact_cr_Ne_default,
    cands_Eact_Ds_Ne,Eact_Ds_Pe_default,
    Rct_default,h_default,
    cands_Elely_res,
    Ratio_Li_SEI_default
]
Target_name = "SEI_LAM_LiP2"
rows_per_file = 3; 
num_scans = Get_Scan_files(
    BasicPath,Target_name,options_SEI_LAM_LiP,
    parameter_names,Pack,
    rows_per_file,Bundle=True)

Total cases number is 9
Combinations saved to 'SEI_LAM_LiP2.csv' file.
CSV files created in folder 'SEI_LAM_LiP2'.


# Full

In [22]:
# Full, everything!
options_Full = {
    "SEI": "interstitial-diffusion limited",
    "SEI on cracks": "true",

    "lithium plating": "partially reversible",
    "lithium plating porosity change":"true",
    "particle mechanics": ("swelling and cracking", "swelling only"),
    "loss of active material": "stress-driven",

    "contact resistance": "true",
    "open-circuit potential": "current sigmoid",
    "SEI film resistance": "distributed", 
    "SEI porosity change": "true",
    "thermal": "lumped",
}
Exps  = [2,3]
Temps = [10.0,25.0,40.0]
cands_Dint  = [6e-19]
cands_Decay =  [1e-6]
cands_LiP =  [1e-10]
cands_LAM_Ne = [6e-9]
cands_LAM_Pe = [1e-6]
cands_k_cr_Ne=[5.29e-25]
cands_V_SEI = [6.41e-5,]
cands_Eact_Ds_Ne = [6e4]
cands_Eact_Ds_Pe = [1.2e4]
cands_Elely_res = [1.0,]
Pack = [
    Exps,Temps,
    cands_Dint,    
    cands_Decay, cands_LiP,cands_LAM_Ne,
    cands_LAM_Pe,cands_k_cr_Ne,
    cands_V_SEI,
    Eact_SEI_default,Eact_cr_Ne_default,
    cands_Eact_Ds_Ne,cands_Eact_Ds_Pe,
    Rct_default,h_default,
    cands_Elely_res,
    Ratio_Li_SEI_default
]
Target_name = "Full_Exp23"
rows_per_file = 3; 
num_scans = Get_Scan_files(
    BasicPath,Target_name,options_Full,
    parameter_names,para_short_name,Pack,
    rows_per_file,Bundle=True)


Total cases number is 6
Combinations saved to 'Full_Exp23_Paper.csv' file.
CSV files created in folder 'Full_Exp23_Paper'.


In [14]:
Eact_cr_Ne_default
Eact_Ds_Pe_default
Rct_default
Eact_Ds_Pe_default
Ratio_Li_SEI_default

[2.0]